In [1]:
import tensorflow as tf
import numpy as np
import efficientnet.keras as efn 
from efficientnet.keras import preprocess_input

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Concatenate, GlobalAvgPool2D, GlobalMaxPool2D, Input, Flatten, Dense
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras import backend as K

from keras.models import load_model

import random
import cv2 as cv
import matplotlib.pyplot as plt
from os import path

Using TensorFlow backend.


In [2]:
batch_size         = 5
matcher_resolution = 224

In [3]:
def create_base_network(input_shape):
    _input = Input(shape=input_shape)
    base_model = efn.EfficientNetB0(weights=None, include_top=False, input_tensor=_input)
    base_model.layers.pop()
    base_model.layers.pop()
    x = GlobalAvgPool2D()(base_model.layers[-1].output)
    return Model(_input, x)

input_shape=(matcher_resolution,matcher_resolution,3)
base_model= create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
input_c = Input(shape=input_shape)

processed_a = base_model(input_a)
processed_b = base_model(input_b)
processed_c = base_model(input_c)

concat_v = Concatenate(axis=-1, name='out_concacat')([processed_a, processed_b, processed_c])

model = Model([input_a, input_b, input_c], [concat_v])

In [4]:
with open(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_matcher.txt') as f:
    lines = f.readlines()

with open(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\data_for_matcher_validation.txt') as f:
    lines_val = f.readlines()

num_val = int(len(lines_val))-1
num_train = len(lines)-1
print('train on', num_train, 'samples;  valid on', num_val, 'samples')



def get_random_data(apply_random=True):
    x1 = np.empty((batch_size, matcher_resolution, matcher_resolution, 3))
    x2 = np.empty((batch_size, matcher_resolution, matcher_resolution, 3))
    x3 = np.empty((batch_size, matcher_resolution, matcher_resolution, 3))
    
    #here load images from line. chose line as randoms
    for i in range(0, batch_size):
        while True:
            if apply_random: #training
                index1 = random.randint(0, num_train)
                index3 = random.randint(0, num_train)
                rec1s = lines[index1].rstrip('\n')
                rec3s = lines[index3].rstrip('\n')
            else: #valid
                index1 = random.randint(0, num_val)
                index3 = random.randint(0, num_val)
                rec1s = lines_val[index1].rstrip('\n')
                rec3s = lines_val[index3].rstrip('\n')

            rec1 = rec1s.split('_')
            rec3 = rec3s.split('_')

            if rec1[-3] == rec3[-3]: #the same object
                continue
                
            inc = random.randint(1, 6)

            rec2 = rec1.copy()
            rec2[-2] = str(int(rec2[-2])+inc)
            rec2s = "_".join(rec2)

            if path.exists(rec2s) == False: #we do not have a pair
                continue
        
            
            x1[i, ...]  = cv.cvtColor(cv.imread(rec1s), cv.COLOR_BGR2RGB)
            x2[i, ...]  = cv.cvtColor(cv.imread(rec2s), cv.COLOR_BGR2RGB)
            x3[i, ...]  = cv.cvtColor(cv.imread(rec3s), cv.COLOR_BGR2RGB)
            break

    y = np.empty((batch_size, 1280*3))

    return x1, x2, x3, y



def data_generator(is_random):
    """data generator for fit_generator"""
    while True:
        x1, x2, x3, y = get_random_data(apply_random=is_random)    
        yield [x1, x2, x3], y


def data_generator_wrapper(is_random):
    return data_generator(is_random)

train on 144034 samples;  valid on 5736 samples


In [5]:
def triplet_loss(y_true, y_pred, alpha = 0.5):
    
    total_lenght = y_pred.shape.as_list()[-1]
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [6]:
#for the second and third round load the best model and reduce lr to half
#model = load_model('D:/data-petr/signate_3rd_ai_edge/matcher/matcher_it3.h5', custom_objects={'triplet_loss': triplet_loss})

model.compile(optimizer=Adadelta(1.0), loss=triplet_loss)


checkpoint      = ModelCheckpoint(r'D:\SIGNATE\Signate_3rd_AI_edge_competition\matcher_model/matcher_ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5', monitor='val_loss', save_weights_only=True, save_best_only=False,verbose=1)
reduce_lr       = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, delta=0.03)

model.fit_generator(data_generator_wrapper(True),
                      steps_per_epoch=1000,
                      validation_data=data_generator_wrapper(False),
                      validation_steps=1000,
                      epochs=50,
                      callbacks=[reduce_lr, checkpoint])





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
1000/1000 [==============================] - 558s 558ms/step - loss: 1.9374 - val_loss: 2.8622

Epoch 00001: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\matcher_model/matcher_ep001-loss1.937-val_loss2.862.h5
Epoch 2/50
1000/1000 [==============================] - 489s 489ms/step - loss: 0.1472 - val_loss: 0.1243

Epoch 00002: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\matcher_model/matcher_ep002-loss0.147-val_loss0.124.h5
Epoch 3/50
1000/1000 [==============================] - 495s 495ms/step - loss: 0.1029 - val_loss: 0.1809

Epoch 00003: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\matcher_model/matcher_ep003-loss0.103-val_loss0.181.h5
Epoch 4/50
1000/1000 [==============================] - 495s 495ms/step - loss: 0.0846 - val_loss: 0.0971

Epoch 00004: saving model to D:\SIGNATE\Signate_3rd_AI_edge_competition\matcher_model/matcher_